In [20]:
from common.inputreader import InputReader, PuzzleWrapper

puzzle = PuzzleWrapper(year=int("2022"), day=int("19"))

puzzle.header()

# Not Enough Minerals

[Open Website](https://adventofcode.com/2022/day/19)

In [21]:

# helper functions
class Blueprint:
    def __init__(self, id: int, costs: dict):
        self.id = id
        self.costs = costs
        self.useful = {
            "ore": max(self.costs["clay"]["ore"],
                       self.costs["obsidian"]["ore"],
                       self.costs["geode"]["ore"]),
            "clay": self.costs["obsidian"]["clay"],
            "obsidian": self.costs["geode"]["obsidian"],
            "geode": float("inf")
        }

    def __repr__(self):
        return (f"Blueprint(costs={self.costs})")


def domain_from_input(input: InputReader) -> set:
    lines = input.lines_as_str()
    # remove empty lines
    lines = [line for line in lines if line]
    blueprints = set()

    for i in range(0, len(lines), 5):
        costs = {}
        # parse id
        line = lines[i].strip(":").split()
        id = int(line[-1])

        # parse ore robot using pattern
        line = lines[i + 1].strip(".").split()
        costs[line[1]] = {
            line[-1]: int(line[-2]),
        }

        # parse clay robot using pattern
        line = lines[i + 2].strip(".").split()
        costs[line[1]] = {
            line[-1]: int(line[-2]),
        }

        # parse obsidian robot using pattern
        line = lines[i + 3].strip(".").split()
        costs[line[1]] = {
            line[-1]: int(line[-2]),
            line[-4]: int(line[-5]),
        }

        # parse geode robot using pattern
        line = lines[i + 4].strip(".").split()
        costs[line[1]] = {
            line[-1]: int(line[-2]),
            line[-4]: int(line[-5]),
        }

        for type in costs.keys():
            costs[type] = costs[type]

        blueprint = Blueprint(id, costs)
        blueprints.add(blueprint)

    return blueprints


test_input = domain_from_input(puzzle.get_code_block(0))
print(test_input)

{Blueprint(costs={'ore': {'ore': 2}, 'clay': {'ore': 3}, 'obsidian': {'clay': 8, 'ore': 3}, 'geode': {'obsidian': 12, 'ore': 3}}), Blueprint(costs={'ore': {'ore': 4}, 'clay': {'ore': 2}, 'obsidian': {'clay': 14, 'ore': 3}, 'geode': {'obsidian': 7, 'ore': 2}})}


In [22]:
# test case (part 1)
class State:
    def __init__(self, robots: dict = None, resources: dict = None, ignored: list = None):
        self.robots = robots.copy() if robots else {
            "ore": 1,
            "clay": 0,
            "obsidian": 0,
            "geode": 0,
        }
        self.resources = resources.copy() if resources else {
            "ore": 0,
            "clay": 0,
            "obsidian": 0,
            "geode": 0
        }
        self.ignored = ignored.copy() if ignored else []

    def copy(self) -> "State":
        return State(self.robots, self.resources, self.ignored)

    def __gt__(self, other):
        return self.resources["geode"] > other.resources["geode"]

    def __repr__(self):
        return f"{{robots: {self.robots}, resources: {self.resources}}}"


def evaluate_options(
        blueprint: Blueprint,
        prior_states: list[State],
        timelimit: int = 26
) -> [tuple[int, list]]:
    time_remaining = timelimit - len(prior_states)
    curr_state = prior_states[-1]

    # determine options for what to build in the next state
    options: list[str] = []
    if time_remaining >= 0:
        # look for something affordable and useful and not ignored last time
        for robot, cost in blueprint.costs.items():
            if (curr_state.robots[robot] < blueprint.useful[robot]
                    and all(curr_state.resources[k] >= v for k, v in cost.items())
                    and robot not in curr_state.ignored):
                options.append(robot)

        # geodes before anything else, don't bother with other types at the end
        if "geode" in options:
            options = ["geode"]
        elif time_remaining < 1:
            options = []
        else:
            # cutting off plans that build resources more than 2 phases back
            if ((curr_state.robots["clay"] > 3 or curr_state.robots["obsidian"]
                 or "obsidian" in options) and "ore" in options):
                options.remove("ore")
            if ((curr_state.robots["obsidian"] > 3 or curr_state.robots["geode"]
                 or "geode" in options) and "clay" in options):
                options.remove("clay")

        # add new resources
        next_state = curr_state.copy()
        for r, n in next_state.robots.items():
            next_state.resources[r] += n

        # the 'do nothing' option
        next_state.ignored += options
        results = [evaluate_options(blueprint, prior_states + [next_state], timelimit)]

        # the rest of the options
        for opt in options:
            next_state_opt = next_state.copy()
            next_state_opt.ignored = []
            next_state_opt.robots[opt] += 1
            for r, n in blueprint.costs[opt].items():
                next_state_opt.resources[r] -= n
            results.append(
                evaluate_options(blueprint, prior_states + [next_state_opt], timelimit)
            )

        return max(results)

    return prior_states[-1].resources["geode"], prior_states


def part_1(reader: InputReader, debug: bool) -> int:
    blueprints = domain_from_input(reader)
    result = 0
    for bp in blueprints:
        r = evaluate_options(bp, [State()], 24)
        result += r[0] * bp.id
    return result


result = part_1(puzzle.example(0), True)
print(result)
assert result == 33

33


In [23]:
# real case (part 1)
def generate_input() -> InputReader:
    lines = puzzle.input().lines_as_str()
    final_lines = []
    # add newline before the word Each.
    for line in lines:
        for next in line.replace("Each", "\nEach").split("\n"):
            final_lines.append(next.strip())
    return InputReader("\n".join(final_lines))


result = part_1(generate_input(), False)
print(result)
assert result == 1144

1144


In [ ]:
# test case (part 2)
def part_2(reader: InputReader, debug: bool) -> int:
    blueprints = list(domain_from_input(reader))
    blueprints.sort(key=lambda x: x.id)
    if len(blueprints) > 3:
        blueprints = blueprints[:3]
    result = 1
    for blueprint in blueprints:
        r = evaluate_options(blueprint, [State()], 32)
        result *= r[0]
    return result


result = part_2(puzzle.example(0), True)
print(result)
assert result == 62

In [25]:
# real case (part 2)
input = generate_input()
result = part_2(input, False)
print(result)
assert result == 19440

19440


In [26]:
# print easters eggs
puzzle.print_easter_eggs()

## Easter Eggs

<span title="If You Give A Mouse An Ore-Collecting Robot">kickstart</span> (If You Give A Mouse An Ore-Collecting Robot)